# Credit Risk Resampling Techniques

In [59]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [61]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [63]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [ ]:
# Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(['loan_status'], axis = 1)


# Create our target
y = df[['loan_status']]

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [7]:
y.head()

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk


In [8]:
y.columns

Index(['loan_status'], dtype='object')

In [9]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

## Below format X to have numerical values

### This is done by:
1. Finding all object columns
2. Separating out datetime columns from the object columns
3. Converting the non-datetime objects to integers using get_dummies from pandas
4. Converting datetime strings to float using to_numeric from pandas
5. Merging the datetime columns back into the new non-datetime columns
6. Droping the original object columns from X and adding into X all the new colums above

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 85 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   pymnt_plan                  68817 non-null  object 
 8   dti                         68817 non-null  float64
 9   delinq_2yrs                 68817 non-null  float64
 10  inq_last_6mths              68817 non-null  float64
 11  open_acc                    68817 non-null  float64
 12  pub_rec                     68817 non-null  float64
 13  revol_bal                   688

In [11]:
the_objs =[item for item in columns if df[item].dtypes=='object' ]
the_objs

['home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'initial_list_status',
 'next_pymnt_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag']

In [12]:
len(the_objs)

10

In [13]:
# Place the objects into a dataframe to convert to numerical values
# This is done for the X dataframe and then separately for the y dataframe
#
X_obj = X[['home_ownership','verification_status','issue_d','pymnt_plan','initial_list_status',
               'next_pymnt_d','application_type','hardship_flag','debt_settlement_flag']]
X_obj.head()

,home_ownership,verification_status,issue_d,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,RENT,Source Verified,Mar-2019,n,w,May-2019,Individual,N,N
1,MORTGAGE,Verified,Mar-2019,n,w,May-2019,Individual,N,N
2,MORTGAGE,Verified,Mar-2019,n,w,May-2019,Individual,N,N
3,RENT,Verified,Mar-2019,n,w,May-2019,Individual,N,N
4,MORTGAGE,Not Verified,Mar-2019,n,w,May-2019,Individual,N,N


In [14]:
len(X_obj.columns)

9

In [15]:
# change columns: home_ownership, verification_status, pymnt_plan, initial_list_status, applicaiton_type,
#hardship_flag, debt_settlement_flag
#Name the dataframe X_7 becaus it has 7 columns to start with.
#Here, columns 'issue_d' and next_pymnt_d are not included in X_7.  They will be handeled separately.
#
columns7 = ['home_ownership','verification_status','pymnt_plan','initial_list_status',
               'application_type','hardship_flag','debt_settlement_flag']

# Make a copy of X_obj and drop the datetime columns
#
X_9 = X_obj.copy()

#Create X_7 which will not have the datetime columns
#
X_7 = X_9.drop(['issue_d', 'next_pymnt_d'], axis=1)

#Convert categorical data to numerical data as integers 
#
X_7 = pd.get_dummies(X_7, columns7)

#Re-name X_7 as X_14 because there are 14 columns now
#
X_14=X_7
X_14

,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,0,0,0,1,0,1,0,1,0,1,1,0,1,1
1,0,1,0,0,0,0,1,1,0,1,1,0,1,1
2,0,1,0,0,0,0,1,1,0,1,1,0,1,1
3,0,0,0,1,0,0,1,1,0,1,1,0,1,1
4,0,1,0,0,1,0,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,0,0,0,1,0,1,0,1,0,1,1,0,1,1
68813,0,0,0,1,1,0,0,1,0,1,1,0,1,1
68814,0,1,0,0,0,1,0,1,0,1,1,0,1,1
68815,0,1,0,0,0,0,1,1,1,0,1,0,1,1


In [16]:
len(X_14.columns)

14

In [17]:
X_14.columns

Index(['home_ownership_ANY', 'home_ownership_MORTGAGE', 'home_ownership_OWN',
       'home_ownership_RENT', 'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'pymnt_plan_n', 'initial_list_status_f', 'initial_list_status_w',
       'application_type_Individual', 'application_type_Joint App',
       'hardship_flag_N', 'debt_settlement_flag_N'],
      dtype='object')

In [18]:
# Now change the datetime columns to numerical values as floats
#
X_2=X_obj[['issue_d', 'next_pymnt_d']]
X_2
X_2.head()

,issue_d,next_pymnt_d
0,Mar-2019,May-2019
1,Mar-2019,May-2019
2,Mar-2019,May-2019
3,Mar-2019,May-2019
4,Mar-2019,May-2019


In [19]:
X_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   issue_d       68817 non-null  object
 1   next_pymnt_d  68817 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [20]:
# Reformat column 'issue_d' and 'next_pymnt_d'
#
X_2['issue_d'] = pd.to_datetime(X_2['issue_d'])
X_2['issue_d'] = X_2['issue_d'].dt.strftime('%Y.%m')
X_2['issue_d'] = pd.to_numeric(X_2['issue_d'])

X_2['next_pymnt_d'] = pd.to_datetime(X_2['next_pymnt_d'])
X_2['next_pymnt_d'] = X_2['next_pymnt_d'].dt.strftime('%Y.%m')
X_2['next_pymnt_d'] = pd.to_numeric(X_2['next_pymnt_d'])

X_2.head()


,issue_d,next_pymnt_d
0,2019.03,2019.05
1,2019.03,2019.05
2,2019.03,2019.05
3,2019.03,2019.05
4,2019.03,2019.05


In [21]:
# Add dataframe X_2 to dataframe X_14 and call it X_16
#
X_16 = X_14.merge(X_2, on=None, left_index=True, right_index=True)
X_16.sample(20)

,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,issue_d,next_pymnt_d
20642,0,1,0,0,0,1,0,1,0,1,1,0,1,1,2019.02,2019.04
5195,0,0,0,1,0,1,0,1,1,0,1,0,1,1,2019.03,2019.05
50682,0,0,0,1,1,0,0,1,0,1,1,0,1,1,2019.01,2019.04
26227,0,1,0,0,0,1,0,1,1,0,1,0,1,1,2019.02,2019.05
11694,0,0,0,1,0,0,1,1,0,1,1,0,1,1,2019.03,2019.05
66634,0,1,0,0,0,1,0,1,0,1,1,0,1,1,2019.01,2019.05
9474,0,1,0,0,1,0,0,1,0,1,1,0,1,1,2019.03,2019.05
12226,0,0,0,1,0,1,0,1,0,1,0,1,1,1,2019.03,2019.05
49486,0,0,0,1,1,0,0,1,1,0,1,0,1,1,2019.01,2019.05
13722,0,0,0,1,0,1,0,1,0,1,1,0,1,1,2019.02,2019.05


In [22]:
len(X_16.columns)

16

In [23]:
new_cols = list(X_16.columns)
new_cols

['home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'verification_status_Not Verified',
 'verification_status_Source Verified',
 'verification_status_Verified',
 'pymnt_plan_n',
 'initial_list_status_f',
 'initial_list_status_w',
 'application_type_Individual',
 'application_type_Joint App',
 'hardship_flag_N',
 'debt_settlement_flag_N',
 'issue_d',
 'next_pymnt_d']

In [24]:
# Update X by replacing the columns in the_objs list minus loan_status with columns in the new_cols list
#
X.drop(['home_ownership',
 'verification_status',
 'issue_d',
 'pymnt_plan',
 'initial_list_status',
 'next_pymnt_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag'], axis=1, inplace=True)

X[new_cols]=X_16[new_cols]
X.columns





Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

In [25]:
len(X.columns)

92

In [26]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,issue_d,next_pymnt_d
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,0,1,1,0,1,1,2019.03,2019.05
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,1,0,1,1,2019.03,2019.05
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,1,0,1,1,2019.03,2019.05
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,1,0,1,1,2019.03,2019.05
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,1,0,1,1,0,1,1,2019.03,2019.05


## Now for y vector to numerical values

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y['loan_status']=le.fit_transform(y['loan_status'])
y

,loan_status
0,1
1,1
2,1
3,1
4,1
...,...
68812,1
68813,1
68814,1
68815,1


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [29]:
# Counter for 
Counter(y_train)

Counter({'loan_status': 1})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [30]:
# Resample the training data with the RandomOversampler
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
#
ros = RandomOverSampler(random_state = 1)
X_resample, y_resample = ros.fit_resample(X_train, y_train)
Counter(y_resample)

Counter({'loan_status': 1})

In [31]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resample, y_resample)


LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
#
from sklearn.metrics import balanced_accuracy_score

# Predict
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5876351168965115

In [33]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  61,   26],
       [9002, 8116]])

In [34]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.70      0.47      0.01      0.58      0.34        87
          1       1.00      0.47      0.70      0.64      0.58      0.32     17118

avg / total       0.99      0.48      0.70      0.64      0.58      0.32     17205



### SMOTE Oversampling

In [39]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

In [41]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_resample, y_resample)

LogisticRegression(random_state=1)

In [42]:
# Calculated the balanced accuracy score
y_pred = model_smote.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5876351168965115

In [43]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  61,   26],
       [9002, 8116]])

In [44]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.70      0.47      0.01      0.58      0.34        87
          1       1.00      0.47      0.70      0.64      0.58      0.32     17118

avg / total       0.99      0.48      0.70      0.64      0.58      0.32     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [52]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [46]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [48]:
# Calculated the balanced accuracy score
y_pred = model_cc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5176069956609497

In [49]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   63,    24],
       [11793,  5325]])

In [50]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.72      0.31      0.01      0.47      0.23        87
          1       1.00      0.31      0.72      0.47      0.47      0.22     17118

avg / total       0.99      0.31      0.72      0.47      0.47      0.22     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [64]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [65]:
# Train the Logistic Regression model using the resampled data
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1)
model_smoteenn.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [66]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5176069956609497

In [67]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   63,    24],
       [11793,  5325]])

In [68]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.72      0.31      0.01      0.47      0.23        87
          1       1.00      0.31      0.72      0.47      0.47      0.22     17118

avg / total       0.99      0.31      0.72      0.47      0.47      0.22     17205

